# Video Actor Synchroncy and Causality (VASC)
## RAEng: Measuring Responsive Caregiving Project
### Caspar Addyman, 2020
### https://github.com/infantlab/VASC

# Step 3: Analyse the data using statsmodels

This script correlates and compares the timeseries of wireframes for the two figures in the video `["parent", "infant"]`

We start by reloading the saved parquet file containing the multi-index numpy array of all [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) data from all pairs of individuals. 



In [1]:
import sys
import os
import json
import math
import numpy as np       
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
%matplotlib inline

import logging
import ipywidgets as widgets  #let's us add buttons and sliders to this page.
from ipycanvas import Canvas

import vasc #a module of our own functions (found in vasc.py in this folder)

#turn on debugging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
%pdb on

Automatic pdb calling has been turned ON


In [2]:
jupwd =  os.getcwd() + "\\"
projectpath = os.getcwd() + "\\..\\SpeakNSign\\"
# projectpath = os.getcwd() + "\\..\\lookit\\"

# locations of videos and output
videos_in = projectpath 
videos_out   = projectpath + "out"
#videos_out = "E:\\SpeakNSign\\out"
videos_out_openpose   = videos_out + "\\openpose"
videos_out_timeseries = videos_out + "\\timeseries"
videos_out_analyses   = videos_out + "\\analyses"

### 3.1 Load the clean data as a DataFrame

Reload the clean data file created in step 2. 

In [3]:
#retrieve the list of base names of processed videos.
try:
    with open(videos_out + '\\clean.json') as json_file:
        videos = json.load(json_file)
        print("Existing clean.json found..")
except:
    print("File clean.json not found.")

Existing clean.json found..


In [4]:
print('reading parquet file:')
df = pq.read_table(videos_out_timeseries + '\\cleandata.parquet').to_pandas()

#sort the column names as this helps with indexing
df = df.sort_index(axis = 1)
print(df.head())

reading parquet file:
video     SS002                                                          \
person   infant                                                           
coord        0        1         2        3        4         5        6    
0       595.737  356.101  0.778361    0.000    0.000  0.000000    0.000   
1       595.836  352.914  0.755376  617.643  393.658  0.181467  595.710   
2       597.299  348.278  0.698885  612.964  393.617  0.119238  592.615   
3       594.182  354.494  0.699191  616.096  393.654  0.193101  592.640   
4       594.176  356.053  0.587353  614.562  395.198  0.171983  595.738   

video                               ...  SS098                                \
person                              ... parent                                 
coord        7         8        9   ...     65   66   67   68   69   70   71   
0         0.000  0.000000    0.000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1       381.128  0.356742  565.966  ...    0.0  0.0  0.0 

## 3.2 Process the data 

Next we set all 0 values to as missing value `np.nan` to enable interpolation.
Then use numpy's built in `interpolate` method. 

In [5]:
df = df.replace(0.0, np.nan)

#are we going to use all the data or a subset?
first = 501
last = 8500

df = df.truncate(before  = first, after = last)

In [6]:
df = df.interpolate()

In [28]:
#take a quick look
print(df.head())
df.shape

video     SS002                                                          \
person   infant                                                           
coord        0        1         2        3        4         5        6    
501     598.880  345.068  0.822972  619.273  393.675  0.069144      NaN   
502     597.376  343.620  0.880787  608.302  382.741  0.152672  602.001   
503     598.915  343.619  0.882141  608.276  382.697  0.192874  598.899   
504     598.909  343.615  0.877841  609.828  382.712  0.191905  603.567   
505     598.879  343.599  0.867191  609.856  382.755  0.193558  598.883   

video                                  ...  SS098                              \
person                                 ... parent                               
coord        7         8           9   ...     65  66  67  68  69  70  71  72   
501         NaN       NaN         NaN  ...    NaN NaN NaN NaN NaN NaN NaN NaN   
502     370.234  0.226657         NaN  ...    NaN NaN NaN NaN NaN NaN NaN N

### 3.2.1 Mean movements
We create a dictionary of the subsets of OpenPose coordinates we want to average and then call `mean` on the Pandas dataframe. e.g.

```
meanpoints = {
               "headx" : [0, 3, 45, 48, 51, 54],
               "heady" : [1, 4, 46, 49, 52, 55],
               "allx" :  [0, 3, 6, 9, ...],
               "ally" :  [1, 4, 7, 10, ...]
             }
```

Then we call the `vasc.averageCoordinateTimeSeries` function to average across sets of coordinates. For a given set of videos and people. For example

In:
```
videos = "All"
people = "Both"
df2 = vasc.averageCoordinateTimeSeries(df,meanpoints,videos,people)
df2.head
```

Out:
```
person      infant                                          parent   
avgs         headx       heady          xs          ys       headx   
501     565.996600  369.840600  534.895615  398.482538  471.686200   
502     567.231800  369.887600  534.354198  398.706552  471.849400   
503     567.228600  370.159600  534.444328  398.678133  471.711600   
504     566.912600  369.857000  535.369536  398.551636  472.309400
...            ...         ...         ...         ...         ...
```


In [7]:
meanpoints = {"head" : vasc.headxys,
              "headx": vasc.headx,
              "heady": vasc.heady,
              "arms" : vasc.armsxys,
              "armsx": vasc.armsx,
              "armsy": vasc.armsy,
              "all"  : vasc.xys,
              "allx" : vasc.xs,
              "ally" : vasc.ys}

vids = "All"
people = ["infant","parent"]

#average across the points in each group (all points of head etc. )
avgdf = vasc.averageCoordinateTimeSeries(df,meanpoints,vids,people)

In [36]:
avgdf.head

<bound method NDFrame.head of video        SS002                                                  \
person      infant                                                   
avgs          head       headx       heady        arms       armsx   
501     484.209300  612.996200  355.422400  517.427500  638.033000   
502     481.419800  609.866600  352.973000  481.789375  582.808250   
503     481.876100  611.113600  352.638600  480.036875  580.876250   
504     482.180500  611.428200  352.932800  482.967700  581.949000   
505     482.019000  611.399200  352.638800  482.010750  580.758567   
...            ...         ...         ...         ...         ...   
8496    213.605056  207.649612  219.560500  221.371282   45.704099   
8497    213.991723  208.433778  219.549667  221.176277   45.800722   
8498    213.353989  210.000478  216.707500  225.352277   42.657220   
8499    213.080025  209.225383  216.934667  219.736273   46.051213   
8500    213.336351  208.174702  218.498000  219.743971   46.

### 3.2.2 Rolling window of movements

One thing we'd like to know is if mothers move in response to infants. The raw time series are probably too noisy to tell us this so instead we can look at few alternatives

1. **Smoothed** - if we average the signal over a short rolling window we smooth out any high-frequency jitter. 
2. **Variance** - the variance of movement over a short rolling window. First we apply 2 second long (50 frame) rolling window to each coordinate of the body and use the stddev or variance function `std()` or `var()` . Then we take averages as in the step above. However, this time we combine x and y coordinates as this is now a movement index.




In [8]:
win = 50 #2 seconds
halfwin = math.floor(win/2)

smoothdf = df.rolling(window = 5).mean()
smoothdf = smoothdf.truncate(before  = first, after = last)

vardf = df.rolling(window = win, min_periods = halfwin).var()
vardf = vardf.truncate(before  = first + 50, after = last) # cut out the empty bits at the start
 
smoothdf = vasc.averageCoordinateTimeSeries(smoothdf,meanpoints,vids,people)
vardf = vasc.averageCoordinateTimeSeries(vardf,meanpoints,vids,people)

In [12]:
vardf.head

<bound method NDFrame.head of video        SS002                                                    \
person      infant                                parent               
avgs          head         arms         all         head        arms   
551       4.876595    49.045036   27.327818   953.511413  273.915660   
552       5.264417    52.050050   28.700318   964.783640  280.512398   
553       5.217756    53.715195   29.434249   978.342794  284.806456   
554       5.157802    54.770208   29.824070   994.690959  286.177511   
555       5.226668    56.665956   30.815911  1011.315036  283.769366   
...            ...          ...         ...          ...         ...   
8496    107.590143  1320.102496  342.646233    21.180646  652.799826   
8497    101.690034  1334.711457  344.736358    18.030202  652.918164   
8498     95.847229  1343.272986  345.388851    15.245874  652.309503   
8499     90.042579  1346.056627  344.663809    13.993584  650.863551   
8500     83.278718  1342.486671  3

Let's create a widget to plot some graphs of the data

In [9]:
vidlist = [] #used to fill dropdown options
for vid in videos:  
    vidlist.append(vid)
        
pickvid = widgets.Dropdown(
    options= vidlist,
    value= vidlist[0],
    description='Subject:'
)

features = []
for f in meanpoints:
    features.append(f)
    
pickfeature = widgets.Dropdown(
    options= features,
    value= features[0],
    description='Feature:'
)

linetypes = ["Mean point", "Smoothed Mean (5 frames)","Variance over 2 secs"]
picktype = widgets.Dropdown(
    options= linetypes,
    value= linetypes[0],
    description='Line type:'
)

def pickvid_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        updateAll(True)
        
def pickfeature_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        updateAll(True)

def picktype_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        updateAll(True)
        
pickvid.observe(pickvid_change, 'value') 
pickfeature.observe(pickfeature_change, 'value') 
picktype.observe(picktype_change, 'value') 
button_update = widgets.Button(description="Redraw")
output = widgets.Output()


def drawGraphs(vid, feature, linetype):
    """Plot input signals"""
    plt.ion()
    f, axarr = plt.subplots(2, sharex=True)
    axarr[0].set_title('Infant')
    axarr[1].set_title('Parent')
    #axarr[0].set_xlabel('Frames')
    axarr[1].set_xlabel('Frames')

    who = ["infant","parent"]

    if linetype == linetypes[0]:
        usedf = avgdf
    elif linetype == linetypes[1]:
        usedf = smoothdf
    else:
        usedf = vardf
        
    #to select a single column..
    infant = usedf[(vid, people[0], feature)].to_frame()
    parent = usedf[(vid, people[1], feature)].to_frame()
    n  = np.arange(usedf.shape[0])
    
    #selecting multiple columns slightly messier
    #infant = df3.loc[50:,(vid, part[0], ('head','arms', 'all'))]
    #parent = df3.loc[50:,(vid, part[1], ('head','arms', 'all'))]

    axarr[0].plot(n,infant)
    axarr[1].plot(n,parent, color='b')

    plt.show() 

def updateAll(forceUpdate = False):
    output.clear_output(wait = True)
    if forceUpdate:
        logging.debug('forceUpdate')
        #slider.value = 0
        #slider.max = videos[pickvid.value][pickcam.value]["end"]
    with output:
        display(pickvid,pickfeature,picktype,button_update)  
        drawGraphs(pickvid.value,pickfeature.value,picktype.value)
    
#draw everything for first time
updateAll(True)
output

Output()

### 3.3 Movement analysis

First we run some simple correlations between the mother and infant.

In [15]:
infant = vardf[(vid, people[0], 'head')].to_frame()
infant.head
print(type(infant))

<class 'pandas.core.frame.DataFrame'>


In [16]:
vid = "SS003"
vardf[(vid, people[0], 'head')].corr(vardf[(vid, people[1], 'head')]) 

0.1577043484821421

In [ ]:
who = ["infant","parent"]
parts = ["head","arms","all"]
results = pd.DataFrame(columns = ("corrHead","lagHead","corrArms","lagArms","corrAll","lagAll","DyadSynScore"),
                      index = videos)

In [ ]:
#loop through colculate for each pair
for vid in videos:
    thisrow = []
    for part in parts:
        #to select a single column..
        pearson = vardf[(vid, people[0], part)].corr(vardf[(vid, people[1], part)])
 
        thisrow.append(pearson) #this is for correlation
        thisrow.append(None) #this is for maximum lag
    
    thisrow.append(None) #don't have DyadSynScore yet 
    results.loc[vid] = thisrow

In [ ]:
results

## 3.4 Comparing to human coding. 

We have a spreadsheet of syhnchrony scores for each parent infant dyad. Here we see if we can find a measure that correlates with the human scores.

First, load up the spreadsheet..

In [19]:
excelpath = projectpath + "\\SS_CARE.xlsx"

filename, file_format = os.path.splitext(excelpath)
if file_format and file_format == 'xls':
    # use default reader 
    videolist = pd.read_excel(excelpath)
else: 
    #since dec 2020 read_excel no longer supports xlsx (!?) so need to use openpyxl like so..
    videolist = pd.read_excel(excelpath, engine = "openpyxl")
    
videolist = videolist.set_index("subject")

In [20]:
videolist

,camera1,camera2,camera3,Group,Include,camera swapped,cambest,camnext,camworst,Karolina comments,...,MatContIRR,MatUnrIRR,InfCoopIRR,InfCompIRR,InfDiffIRR,InfPassiveIRR,Household_CAT,DS_CATS,filter_$,DS_CATS2
subject,,,,,,,,,,,,,,,,,,,,,
SS003,U:\Toddler_videos\SS003\SS003_13314_1.mp4,U:\Toddler_videos\SS003\SS003_13314_2.mp4,U:\Toddler_videos\SS003\SS003_13314_3.mp4,0,True,True,camera2,camera3,camera1,+ SS003 - Camera 2 -- fully cleaned - there's...,...,0.0,6.0,6.0,0.0,8.0,0.0,1.0,3,0,2
SS010,U:\Toddler_videos\SS010\PCI_SS010_1.mp4,U:\Toddler_videos\SS010\PCI_SS010_2.mp4,U:\Toddler_videos\SS010\PCI_SS010_3.mp4,0,True,False,camera1,camera2,camera3,+ SS010 - Camera 1 -- mostly cleaned - there ...,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,1
SS013,U:\Toddler_videos\ss013\CAM1_20140515-103645.mp4,U:\Toddler_videos\ss013\CAM2_20140515-103645.mp4,U:\Toddler_videos\ss013\CAM3_20140515-103645.mp4,0,True,False,camera1,camera2,camera3,SS013 - Camera 2 -- fully cleaned - after remo...,...,0.0,7.0,9.0,0.0,0.0,5.0,3.0,2,0,1
SS015,U:\Toddler_videos\SS015\SS015_PCI1.mp4,U:\Toddler_videos\SS015\SS015_PCI2.mp4,U:\Toddler_videos\SS015\SS015_PCI3.mp4,0,True,True,camera3,camera2,camera1,SS015 - Camera 1 -- partly cleaned- had many f...,...,0.0,8.0,4.0,1.0,3.0,6.0,3.0,1,0,1
SS016,U:\Toddler_videos\SS016\SS016_PCI1.mp4,U:\Toddler_videos\SS016\SS016_PCI2.mp4,U:\Toddler_videos\SS016\SS016_PCI3.mp4,0,True,False,camera1,camera2,camera3,+ SS016 - Camera 1 -- fully cleaned,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1,0,1
SS017,U:\Toddler_videos\SS017\SS017-pci1.mp4,U:\Toddler_videos\SS017\SS017-pci2.mp4,U:\Toddler_videos\SS017\SS017-pci3.mp4,0,True,False,camera1,camera2,camera3,+ SS017 - Camera 1 -- fully cleaned - start ar...,...,1.0,6.0,5.0,0.0,3.0,6.0,3.0,2,0,1
SS020,U:\Toddler_videos\SS020\SS020_PCI_1.mp4,U:\Toddler_videos\SS020\SS020_PCI_2.mp4,U:\Toddler_videos\SS020\SS020_PCI_3.mp4,0,True,True,camera3,camera1,camera2,SS020 - Camera 3 -- fully cleaned - lots of mi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,1
SS021,U:\Toddler_videos\SS021\SS021_PCI_1.mp4,U:\Toddler_videos\SS021\SS021_PCI_2.mp4,U:\Toddler_videos\SS021\SS021_PCI_3.mp4,0,True,False,camera1,NaN,NaN,+ SS021 - Camera 1 -- fully cleaned,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1
SS022,U:\Toddler_videos\SS022\CAM1_20140624-133628.mp4,U:\Toddler_videos\SS022\CAM2_20140624-133629.mp4,U:\Toddler_videos\SS022\CAM3_20140624-133629.mp4,0,True,True,camera3,camera1,camera2,+ SS022 - Camera 3 -- fully cleaned,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,1


In [ ]:

results["DyadSynScore"] = videolist["DyadSyn"]
results["MatSensScore"] = videolist["MatSens"]

In [ ]:
results

In [ ]:
vid = "SS095"
# Set window size to compute moving window synchrony.
r_window_size = 120
# Compute rolling window synchrony

#pearson = df3[(vid, who[0], part)].corr(df3[(vid, who[1], part)])
rolling_r = df3[(vid, who[0], parts[0])].rolling(window=500, center=True).corr(df3[(vid, who[1], parts[1])])

f,ax=plt.subplots(2,1,figsize=(14,6),sharex=True)
df3.loc[:,(vid, slice(None), parts[0])].plot(ax=ax[0])
ax[0].set(xlabel='Frame',ylabel='Movement index for parent and infant')
rolling_r.plot(ax=ax[1])
ax[1].set(xlabel='Frame',ylabel='Pearson r')
plt.suptitle("Movement rolling window correlation")

In [ ]:
rolling_r.mean()

In [ ]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

vid = "SS095"
d1 = df3[(vid, who[0], parts[0])]
d2 = df3[(vid, who[1], parts[0])]
seconds = 5
fps = 25
wholeads = who[0] + 'leads <> ' + who[1] + ' leads'
rs = [crosscorr(d1,d2, lag) for lag in range(-int(seconds*fps-1),int(seconds*fps))]
offset = np.ceil(len(rs)/2)-np.argmax(rs)
f,ax=plt.subplots(figsize=(14,3))
ax.plot(rs)
ax.axvline(np.ceil(len(rs)/2),color='k',linestyle='--',label='Center')
ax.axvline(np.argmax(rs),color='r',linestyle='--',label='Peak synchrony')
ax.set(title=f'Offset = {offset} frames\n' + wholeads,ylim=[.0,1],xlim=[0,300], xlabel='Offset',ylabel='Pearson r')
ax.set_xticklabels([int(item-150) for item in ax.get_xticks()]);
plt.legend()

## 3.4 Granger Causality

The next thing to look at is if the movements of the infant predict the movements of the parent. This would suggest parent is responding to the infant. 


In [ ]:

https://towardsdatascience.com/granger-causality-and-vector-auto-regressive-model-for-time-series-forecasting-3226a64889a6

https://www.machinelearningplus.com/time-series/time-series-analysis-python/
    
https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9
    